In [1]:
<div style = "text-align: center"><font size = 6 color = "#B22222" face = "verdana"><b>Ground water level prediction</b></font></div> <br/> 
<div style = "text-align: center"><font size = 5 color = "#00008B" face = "verdana"><b>Md.Abdullah-Al Mamun</b></font></div><br/> 
<div style = "text-align: center"><font size = 5 color = "#00008B" face = "verdana"><b>Mst Nazneen Aktar</b></font></div>

SyntaxError: invalid syntax (3699631420.py, line 1)

# 1. Introduction

## About the dataset


Data collected from meteorological stations in Rangpur, Nilphamari, and Dinajpur was obtained from various sources, including the Bangladesh Meteorological Department (BMD), Bangladesh Water Development Board (BWDB), Food and Agricultural Organization (FAO), National Oceanic and Atmospheric Association (NOAA) Climate Prediction Center (CPC), Bangladesh Bureau of Statistics, and National Aeronautics and Space Administration (NASA). Additionally, monthly groundwater level (GWL) data for 12 selected wells was acquired from the Bangladesh Water Development Board (BWDB). The MODIS data was originally in Network Common Data Form (NetCDF) format and was subsequently converted to Excel format. This comprehensive dataset covers the period from 1993 to 2017.

## Data


**1. Groundwater Level (m) Data for 12 Selected Wells from the Bangladesh Water Development Board (BWDB)**
- Dinajpur-34 (Dinajpur Sadar)

**2. Monthly Temperature (°C)**
- Dinajpur_Tave

**3. Monthly Rainfall (mm)**
- Dinajpur_PRCP

**4. MODIS Normalized Difference Vegetation Index (NDVI)**
- Dinajpur_NDVI

**5. Indian Ocean Dipole (IOD)**
- IOD_Value
    
**6. Southern Oscillation Index (SOI)**
- SOI_Value
    
**7. Nina3.4 Value**
- Nina3.4_Value
    
**8. MEI Value**
- MEI_Value


## Identify the Features and Targets

### Features (Independent Variables):

1. Temperature (°C) - Dinajpur_Tave (converted to monthly(mean)-->convert to daily)
2. Rainfall (mm) - Dinajpur_PRCP Rainfall (mm) - Dinajpur_PRCP (averaged monthly, with the same value applied for each day, considering it as the mean monthly value)
3. MODIS Normalized Difference Vegetation Index (NDVI) - Dinajpur_NDVI(monthly-->coverted daily with daily values assumed to be the mean value for the entire month)
4. Indian Ocean Dipole (IOD) - IOD_Value
5. Southern Oscillation Index (SOI) - SOI_Value
6. Nina3.4 Value - Nina3.4_Value
7. MEI Value - MEI_Value

### Target (Dependent Variable):

1. Groundwater Level (m) - Dinajpur-34 (Dinajpur Sadar)

### Libraries

In [ ]:
# Libraries
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from scipy import stats
from tbats import TBATS
import matplotlib.pyplot as plt
from skopt import BayesSearchCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from keras.models import Sequential
from skopt.space import Integer, Real
from sklearn.decomposition import PCA
from catboost import CatBoostRegressor
from keras.layers import LSTM, GRU, Dense
from sklearn.pipeline import make_pipeline
from statsmodels.tsa.arima.model import ARIMA
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.tsa.vector_ar.var_model import VAR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.linear_model import LinearRegression, Ridge
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.gaussian_process import GaussianProcessRegressor
from statsmodels.nonparametric.kernel_regression import KernelReg
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.gaussian_process.kernels import RBF, Matern, WhiteKernel
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, explained_variance_score

warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

### Loading dataset...

In [ ]:
GWL_df= pd.read_excel('Final Dataset_GWL\Final_GWL_Three_location merged all data.xlsx')  # Specify the header row number

GWL_df = GWL_df.rename(columns={'Sayedpur_Tave': 'Saidpur_Tave','Sayedpur_PRCP': 'Saidpur_PRCP', 'Sayedpur_NDVI': 'Saidpur_NDVI'})

GWL_df

In [ ]:
GWL_df.columns

In [ ]:
column_names = ['Year', 'Month', 'Region', 'Well', 'Tave', 'PRCP', 'NDVI', 'IOD', 'SOI', 'Nina', 'MEI', 'Water_Level']

df = pd.DataFrame(columns=column_names)

for index, row in GWL_df.iterrows():
    Year = row['Year']
    Month = row['Month']
    IOD = row['IOD_Value']
    SOI = row['SOI_Value']
    Nina = row['Nina3.4_Value']
    MEI = row['MEI_Value']
    
    for column in GWL_df.columns[2:14]:
        Region = column.split('-')[0]
        
        Well = column.split('-')[1].split()[0]
        PRCP = GWL_df.loc[index, Region+'_PRCP']
        NDVI = GWL_df.loc[index, Region+'_NDVI']
        Tave = GWL_df.loc[index, Region+'_Tave']
        Water_Level = GWL_df.loc[index, column]

        # Create a new row for the target DataFrame
        new_row = {'Year':Year, 'Month':Month, 'Region':Region, 'Well':Well, 'PRCP':PRCP, 'Tave': Tave,
                   'NDVI':NDVI, 'IOD':IOD, 'SOI':SOI, 'Nina':Nina, 'MEI':MEI, 'Water_Level':Water_Level}
        
        df0 = pd.DataFrame(new_row, index=[0])
        df = pd.concat([df, df0], axis=0, ignore_index=True)
        
#         df = df.append(new_row, ignore_index=True)

In [ ]:
df

# Yazid's Prediction

In [ ]:
label_encoder = LabelEncoder()
df['Region'] = label_encoder.fit_transform(df['Region'])
df['Region'] = df['Region'].astype(float)
df['Well'] = df['Well'].astype(float)

In [ ]:
df_train = df.loc[df['Year'] <= 2012].drop(columns=['Year'])
df_test = df.loc[df['Year'] >= 2013].drop(columns=['Year'])

X_train = df_train.drop(columns=['Water_Level'], axis=1).to_numpy()
y_train = df_train['Water_Level'].to_numpy()

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


X_test = df_test.drop(columns=['Water_Level'], axis=1).to_numpy()
y_test = df_test['Water_Level'].to_numpy()

In [ ]:
df_train.info()

In [2]:
df_test

NameError: name 'df_test' is not defined

In [ ]:
X_val.shape

## Modeling

In [ ]:
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Ridge': Ridge(),
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'Gaussian Process': GaussianProcessRegressor(),
    'Gaussian Process Regressor': GaussianProcessRegressor(random_state=42),
    'Weighted K-Nearest Neighbors': KNeighborsRegressor(weights='distance'),
    'LightGBM': LGBMRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(random_state=42, verbose=0),
    'XGBoost': XGBRegressor(),
    'LGBM Regressor': lgb.LGBMRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42)
}

results = {}

for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on the training set
    y_pred_train = model.predict(X_train)
    
    # Predict on the validation set
    y_pred_val = model.predict(X_val)
    
    # Predict on the test set
    y_pred_test = model.predict(X_test)

    # Calculate metrics for the training set
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    train_mae = mean_absolute_error(y_train, y_pred_train)
    train_r2 = r2_score(y_train, y_pred_train)
    train_rae = mean_absolute_percentage_error(y_train, y_pred_train)
    train_rrse = np.sqrt(np.sum((y_train - y_pred_train)**2) / np.sum((y_train - np.mean(y_train))**2))
    train_cc = np.corrcoef(y_train, y_pred_train)[0, 1]

    # Calculate metrics for the validation set
    val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    val_mae = mean_absolute_error(y_val, y_pred_val)
    val_r2 = r2_score(y_val, y_pred_val)
    val_rae = mean_absolute_percentage_error(y_val, y_pred_val)
    val_rrse = np.sqrt(np.sum((y_val - y_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
    val_cc = np.corrcoef(y_val, y_pred_val)[0, 1]
    
    # Calculate metrics for the test set
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    test_mae = mean_absolute_error(y_test, y_pred_test)
    test_r2 = r2_score(y_test, y_pred_test)
    test_rae = mean_absolute_percentage_error(y_test, y_pred_test)
    test_rrse = np.sqrt(np.sum((y_test - y_pred_test)**2) / np.sum((y_test - np.mean(y_test))**2))
    test_cc = np.corrcoef(y_test, y_pred_test)[0, 1]

    results[name] = {
        'Train RMSE': train_rmse,
        'Train MAE': train_mae,
        'Train R-squared': train_r2,
        'Train RAE': train_rae,
        'Train RRSE': train_rrse,
        'Train CC': train_cc,        
        'Validation RMSE': val_rmse,
        'Validation MAE': val_mae,
        'Validation R-squared': val_r2,
        'Validation RAE': val_rae,
        'Validation RRSE': val_rrse,
        'Validation CC': val_cc,
        'Test RMSE': test_rmse,
        'Test MAE': test_mae,
        'Test R-squared': test_r2,
        'Test RAE': test_rae,
        'Test RRSE': test_rrse,
        'Test CC': test_cc
    }

# Print the results
for model, metrics in results.items():
    print(f"Metrics for {model}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.3f}")
    print("\n")

### LWLR model


In [ ]:
num_neighbors = 5  

# Initialize LWLR model
lwlr = KNeighborsRegressor(n_neighbors=num_neighbors, weights='distance')

# Train the LWLR model
lwlr.fit(X_train, y_train)

# Predict on validation set
lwlr_pred = lwlr.predict(X_val)

# Calculate metrics for LWLR
lwlr_rmse = np.sqrt(mean_squared_error(y_val, lwlr_pred))
lwlr_mae = mean_absolute_error(y_val, lwlr_pred)
lwlr_r2 = r2_score(y_val, lwlr_pred)
lwlr_rae = mean_absolute_percentage_error(y_val, lwlr_pred)
lwlr_rrse = np.sqrt(np.sum((y_val - lwlr_pred)**2) / np.sum((y_val - np.mean(y_val))**2))
lwlr_cc = np.corrcoef(y_val, lwlr_pred)[0, 1]

# the evaluation metrics for LWLR
print("LWLR RMSE:", lwlr_rmse)
print("LWLR MAE:", lwlr_mae)
print("LWLR R-squared:", lwlr_r2)
print("LWLR RAE:", lwlr_rae)
print("LWLR RRSE:", lwlr_rrse)
print("LWLR CC:", lwlr_cc)

Based on the evaluation metrics provided, here are the 12 best models, ranked by RMSE, MAE, and R-squared values:

1. Random Forest Regressor (RMSE: 4.5129170688300314e-15, MAE: 3.731747590232183e-15, R-squared: 1.0)
2. lwlr
3. Poly Kernel Regression (RMSE: 1.6127342607400202e-14, MAE: 1.2373097451168137e-14, R-squared: 1.0)
4. RBF Kernel Regression (RMSE: 8.402295816463806e-12, MAE: 7.30005059367716e-12, R-squared: 1.0)
5. Gaussian Process Regression (RMSE: 1.3658267901830649e-11, MAE: 1.321696512977017e-11, R-squared: 1.0)
6. Weighted K-Nearest Neighbors (RMSE: 1.6151202582352662e-16, MAE: 6.094448088633645e-17, R-squared: 1.0)
7. K-Nearest Neighbors (RMSE: 1.6151202582352662e-16, MAE: 6.094448088633645e-17, R-squared: 1.0)
8. XGBoost Regressor (RMSE: 0.0001604180170547465, MAE: 0.00012571969879465955, R-squared: 0.9999999418520767)
9. CatBoost Regressor (RMSE: 0.0004191192864975924, MAE: 0.00033127405418021457, R-squared: 0.9999996030797887)
10. LightGBM Regressor (RMSE: 0.0032090330540582293, MAE: 0.002630020414763086, R-squared: 0.9999767310759211)
11. Gradient Boosting Regressor (RMSE: 0.2065918821936626, MAE: 0.16683381944876388, R-squared: 0.9035606162870705)
12. Ridge Regression (RMSE: 0.525636725173341, MAE: 0.429759090595957, R-squared: 0.37569139148812536)

# Hyper parameter tuning

## 1. Random Forest Regressor HPT

In [ ]:
# Initialize Random Forest model
random_forest = RandomForestRegressor()

# the hyperparameters and their possible values for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 200],  
    'max_depth': [None, 5, 10],    
    'min_samples_split': [2, 5, 10] 
}

# Initialize Grid Search for Random Forest
grid_search_rf = GridSearchCV(random_forest, param_grid_rf, scoring='neg_mean_squared_error', cv=5)

# Perform Grid Search for Random Forest
grid_search_rf.fit(X_train, y_train)

# Get the best hyperparameters and model for Random Forest
best_rf = grid_search_rf.best_estimator_
best_params_rf = grid_search_rf.best_params_

# Predict on validation set using Random Forest
rf_pred_val = best_rf.predict(X_val)

# Calculate metrics for Random Forest on validation set
rf_rmse_val = np.sqrt(mean_squared_error(y_val, rf_pred_val))
rf_mae_val = mean_absolute_error(y_val, rf_pred_val)
rf_r2_val = r2_score(y_val, rf_pred_val)
rf_rae_val = mean_absolute_percentage_error(y_val, rf_pred_val)
rf_rrse_val = np.sqrt(np.sum((y_val - rf_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
rf_cc_val = np.corrcoef(y_val, rf_pred_val)[0, 1]

# Print the best hyperparameters and evaluation metrics for Random Forest on validation set
print("Best Hyperparameters for Random Forest:", best_params_rf)
print("Random Forest RMSE (Validation):", rf_rmse_val)
print("Random Forest MAE (Validation):", rf_mae_val)
print("Random Forest R-squared (Validation):", rf_r2_val)
print("Random Forest RAE (Validation):", rf_rae_val)
print("Random Forest RRSE (Validation):", rf_rrse_val)
print("Random Forest CC (Validation):", rf_cc_val)

## 2. LWLR HPT

In [ ]:
# the range of neighbors to consider
param_grid_lwlr = {
    'n_neighbors': [3, 5, 7],  # You can adjust these values
    'weights': ['uniform', 'distance']  # You can adjust these values
}

# Initialize LWLR model
lwlr = KNeighborsRegressor()

# Initialize Grid Search for LWLR
grid_search_lwlr = GridSearchCV(lwlr, param_grid_lwlr, scoring='neg_mean_squared_error', cv=5)

# Perform Grid Search for LWLR
grid_search_lwlr.fit(X_train, y_train)

# Get the best hyperparameters and model for LWLR
best_lwlr = grid_search_lwlr.best_estimator_
best_params_lwlr = grid_search_lwlr.best_params_

# Predict on validation set using LWLR
lwlr_pred_val = best_lwlr.predict(X_val)

# Calculate metrics for LWLR on validation set
lwlr_rmse_val = np.sqrt(mean_squared_error(y_val, lwlr_pred_val))
lwlr_mae_val = mean_absolute_error(y_val, lwlr_pred_val)
lwlr_r2_val = r2_score(y_val, lwlr_pred_val)
lwlr_rae_val = mean_absolute_percentage_error(y_val, lwlr_pred_val)
lwlr_rrse_val = np.sqrt(np.sum((y_val - lwlr_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
lwlr_cc_val = np.corrcoef(y_val, lwlr_pred_val)[0, 1]

# Print the best hyperparameters and evaluation metrics for LWLR on validation set
print("Best Hyperparameters for LWLR:", best_params_lwlr)
print("LWLR RMSE (Validation):", lwlr_rmse_val)
print("LWLR MAE (Validation):", lwlr_mae_val)
print("LWLR R-squared (Validation):", lwlr_r2_val)
print("LWLR RAE (Validation):", lwlr_rae_val)
print("LWLR RRSE (Validation):", lwlr_rrse_val)
print("LWLR CC (Validation):", lwlr_cc_val)

## 3. Gaussian Process Regression HPT

In [ ]:
# the hyperparameters and their possible values for Gaussian Process Regression
param_grid_gpr = {
    'kernel': [None, 1.0 * RBF(length_scale=1.0), Matern(length_scale=1.0, nu=1.5), WhiteKernel(noise_level=1.0)],   
  
}

# Initialize Grid Search for Gaussian Process Regression
grid_search_gpr = GridSearchCV(GaussianProcessRegressor(), param_grid_gpr, scoring='neg_mean_squared_error', cv=5)
grid_search_gpr.fit(X_train, y_train)

# Get the best hyperparameters and model for Gaussian Process Regression
best_gpr = grid_search_gpr.best_estimator_
best_params_gpr = grid_search_gpr.best_params_

# Predict on validation set using Gaussian Process Regression
gpr_pred_val = best_gpr.predict(X_val)

# Calculate metrics for Gaussian Process Regression on validation set
gpr_rmse_val = np.sqrt(mean_squared_error(y_val, gpr_pred_val))
gpr_mae_val = mean_absolute_error(y_val, gpr_pred_val)
gpr_r2_val = r2_score(y_val, gpr_pred_val)
gpr_rae_val = mean_absolute_percentage_error(y_val, gpr_pred_val)
gpr_rrse_val = np.sqrt(np.sum((y_val - gpr_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
gpr_cc_val = np.corrcoef(y_val, gpr_pred_val)[0, 1]

# Print the best hyperparameters and evaluation metrics for Gaussian Process Regression on validation set
print("Best Hyperparameters for Gaussian Process Regression:", best_params_gpr)
print("Gaussian Process RMSE (Validation):", gpr_rmse_val)
print("Gaussian Process MAE (Validation):", gpr_mae_val)
print("Gaussian Process R-squared (Validation):", gpr_r2_val)
print("Gaussian Process RAE (Validation):", gpr_rae_val)
print("Gaussian Process RRSE (Validation):", gpr_rrse_val)
print("Gaussian Process CC (Validation):", gpr_cc_val)   

## 4. Weighted K-Nearest Neighbors HPT

In [ ]:
# the hyperparameters and their possible values for Weighted K-Nearest Neighbors
param_grid_wknn = {
     'n_neighbors': [3, 5, 7, 9, 11],  
    'weights': ['uniform', 'distance'],  
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  
    'leaf_size': [10, 30, 50], 
    'p': [1, 2],
    'metric': ['euclidean', 'manhattan']  
}

# Initialize Grid Search for Weighted K-Nearest Neighbors
grid_search_wknn = GridSearchCV(KNeighborsRegressor(), param_grid_wknn, scoring='neg_mean_squared_error', cv=5)
grid_search_wknn.fit(X_train, y_train)

# Get the best hyperparameters and model for Weighted K-Nearest Neighbors
best_wknn = grid_search_wknn.best_estimator_
best_params_wknn = grid_search_wknn.best_params_

# Predict on validation set using Weighted K-Nearest Neighbors
wknn_pred_val = best_wknn.predict(X_val)

# Calculate metrics for Weighted K-Nearest Neighbors on validation set
wknn_rmse_val = np.sqrt(mean_squared_error(y_val, wknn_pred_val))
wknn_mae_val = mean_absolute_error(y_val, wknn_pred_val)
wknn_r2_val = r2_score(y_val, wknn_pred_val)
wknn_rae_val = mean_absolute_percentage_error(y_val, wknn_pred_val)
wknn_rrse_val = np.sqrt(np.sum((y_val - wknn_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
wknn_cc_val = np.corrcoef(y_val, wknn_pred_val)[0, 1]

# Print the best hyperparameters and evaluation metrics for Weighted K-Nearest Neighbors on validation set
print("Best Hyperparameters for Weighted K-Nearest Neighbors:", best_params_wknn)
print("Weighted K-NN RMSE (Validation):", wknn_rmse_val)
print("Weighted K-NN MAE (Validation):", wknn_mae_val)
print("Weighted K-NN R-squared (Validation):", wknn_r2_val)
print("Weighted K-NN RAE (Validation):", wknn_rae_val)
print("Weighted K-NN RRSE (Validation):", wknn_rrse_val)
print("Weighted K-NN CC (Validation):", wknn_cc_val)

## 5. K-Nearest Neighbors HPT

### The KNeighborsRegressor (KNN) model doesn't have traditional hyperparameters like other models (e.g., Random Forest).However, I performed hyperparameter tuning for the Locally Weighted Linear Regression (LWLR) using Grid Search with a specified range of neighbors and weight options, ultimately finding the best hyperparameters and model for LWLR, and evaluating its performance on the validation set and test set.

In [ ]:
# the range of neighbors and weights to consider
param_grid = {
     'n_neighbors': [3, 5, 7, 9, 11],  
    'weights': ['uniform', 'distance'],  
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  
    'leaf_size': [10, 30, 50], 
    'p': [1, 2],  
    'metric': ['euclidean', 'manhattan'] 
}

# Initialize KNN model
knn = KNeighborsRegressor()

# Initialize Grid Search
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error')

# Perform Grid Search
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the KNN model with the best hyperparameters
best_knn = KNeighborsRegressor(n_neighbors=best_params['n_neighbors'], weights=best_params['weights'])
best_knn.fit(X_train, y_train)

# Predict on validation set
knn_pred = best_knn.predict(X_val)

# Calculate metrics for KNN
knn_rmse = np.sqrt(mean_squared_error(y_val, knn_pred))
knn_mae = mean_absolute_error(y_val, knn_pred)
knn_r2 = r2_score(y_val, knn_pred)
knn_rae = mean_absolute_percentage_error(y_val, knn_pred)
knn_rrse = np.sqrt(np.sum((y_val - knn_pred)**2) / np.sum((y_val - np.mean(y_val))**2))
knn_cc = np.corrcoef(y_val, knn_pred)[0, 1]

# Print the best hyperparameters and evaluation metrics for KNN
print("Best Hyperparameters for KNN:", best_params)
print("KNN RMSE:", knn_rmse)
print("KNN MAE:", knn_mae)
print("KNN R-squared:", knn_r2)
print("KNN RAE:", knn_rae)
print("KNN RRSE:", knn_rrse)
print("KNN CC:", knn_cc)

## 6. XGBoost Regressor HPT

In [ ]:
# Define the search space for hyperparameters
param_dist = {
    'n_estimators': Integer(100, 300),
    'max_depth': Integer(3, 5),
    'learning_rate': Real(0.01, 0.2, prior='uniform'),
    'gamma': Real(0, 0.2, prior='uniform')
}

# Initialize Bayesian Optimization for XGBoost Regressor
bayes_search_xgb = BayesSearchCV(
    XGBRegressor(),
    param_dist,
    scoring='neg_mean_squared_error',
    cv=5,
    n_iter=10, 
    random_state=42,  
    n_jobs=-1,  
    verbose=1, 
    n_points=5, 
    refit=True  
)

# Perform Bayesian Optimization
bayes_search_xgb.fit(X_train, y_train)

# Get the best hyperparameters and model for XGBoost Regressor
best_xgb = bayes_search_xgb.best_estimator_
best_params_xgb = bayes_search_xgb.best_params_

# Predict on validation set using XGBoost Regressor
xgb_pred_val = best_xgb.predict(X_val)

# Calculate metrics for XGBoost Regressor on validation set
xgb_rmse_val = np.sqrt(mean_squared_error(y_val, xgb_pred_val))
xgb_mae_val = mean_absolute_error(y_val, xgb_pred_val)
xgb_r2_val = r2_score(y_val, xgb_pred_val)
xgb_rae_val = mean_absolute_percentage_error(y_val, xgb_pred_val)
xgb_rrse_val = np.sqrt(np.sum((y_val - xgb_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
xgb_cc_val = np.corrcoef(y_val, xgb_pred_val)[0, 1]

# Print the best hyperparameters and evaluation metrics for XGBoost Regressor on validation set
print("Best Hyperparameters for XGBoost Regressor:", best_params_xgb)
print("XGBoost RMSE (Validation):", xgb_rmse_val)
print("XGBoost MAE (Validation):", xgb_mae_val)
print("XGBoost R-squared (Validation):", xgb_r2_val)
print("XGBoost RAE (Validation):", xgb_rae_val)
print("XGBoost RRSE (Validation):", xgb_rrse_val)
print("XGBoost CC (Validation):", xgb_cc_val)

## 7. CatBoost Regressor  HPT

In [ ]:

# Define the search space for hyperparameters
param_dist = {
    'iterations': Integer(100, 300),
    'depth': Integer(4, 8),
    'learning_rate': Real(0.01, 0.2, prior='uniform')
}

# Initialize Bayesian Optimization for CatBoost Regressor
bayes_search_catboost = BayesSearchCV(
    CatBoostRegressor(verbose=0),
    param_dist,
    scoring='neg_mean_squared_error',
    cv=5,
    n_iter=10,
    random_state=42,  
    n_jobs=-1, 
    verbose=1,  
    n_points=5,
    refit=True  
)

# Perform Bayesian Optimization
bayes_search_catboost.fit(X_train, y_train)

# Get the best hyperparameters and model for CatBoost Regressor
best_catboost = bayes_search_catboost.best_estimator_
best_params_catboost = bayes_search_catboost.best_params_

# Predict on validation set using CatBoost Regressor
catboost_pred_val = best_catboost.predict(X_val)

# Calculate metrics for CatBoost Regressor on validation set
catboost_rmse_val = np.sqrt(mean_squared_error(y_val, catboost_pred_val))
catboost_mae_val = mean_absolute_error(y_val, catboost_pred_val)
catboost_r2_val = r2_score(y_val, catboost_pred_val)
catboost_rae_val = mean_absolute_percentage_error(y_val, catboost_pred_val)
catboost_rrse_val = np.sqrt(np.sum((y_val - catboost_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
catboost_cc_val = np.corrcoef(y_val, catboost_pred_val)[0, 1]

# Print the best hyperparameters and evaluation metrics for CatBoost Regressor on validation set
print("Best Hyperparameters for CatBoost Regressor:", best_params_catboost)
print("CatBoost RMSE (Validation):", catboost_rmse_val)
print("CatBoost MAE (Validation):", catboost_mae_val)
print("CatBoost R-squared (Validation):", catboost_r2_val)
print("CatBoost RAE (Validation):", catboost_rae_val)
print("CatBoost RRSE (Validation):", catboost_rrse_val)
print("CatBoost CC (Validation):", catboost_cc_val)

## 8. LightGBM Regressor HPT

In [ ]:
# Define the search space for hyperparameters
param_dist = {
    'n_estimators': Integer(100, 300),
    'max_depth': Integer(3, 5),
    'learning_rate': Real(0.01, 0.2, prior='uniform')
}

# Initialize Bayesian Optimization for LightGBM Regressor
bayes_search_lgb = BayesSearchCV(
    LGBMRegressor(verbosity=-1), 
    param_dist,
    scoring='neg_mean_squared_error',
    cv=5,
    n_iter=10,  
    random_state=42, 
    n_jobs=-1,  
    verbose=1, 
    n_points=5,  
    refit=True 
)

# Perform Bayesian Optimization
bayes_search_lgb.fit(X_train, y_train)

# Get the best hyperparameters and model for LightGBM Regressor
best_lgb = bayes_search_lgb.best_estimator_
best_params_lgb = bayes_search_lgb.best_params_

# Predict on validation set using LightGBM Regressor
lgb_pred_val = best_lgb.predict(X_val)

# Calculate metrics for LightGBM Regressor on validation set
lgb_rmse_val = np.sqrt(mean_squared_error(y_val, lgb_pred_val))
lgb_mae_val = mean_absolute_error(y_val, lgb_pred_val)
lgb_r2_val = r2_score(y_val, lgb_pred_val)
lgb_rae_val = mean_absolute_percentage_error(y_val, lgb_pred_val)
lgb_rrse_val = np.sqrt(np.sum((y_val - lgb_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
lgb_cc_val = np.corrcoef(y_val, lgb_pred_val)[0, 1]

# Print the best hyperparameters and evaluation metrics for LightGBM Regressor on validation set
print("Best Hyperparameters for LightGBM Regressor:", best_params_lgb)
print("LightGBM RMSE (Validation):", lgb_rmse_val)
print("LightGBM MAE (Validation):", lgb_mae_val)
print("LightGBM R-squared (Validation):", lgb_r2_val)
print("LightGBM RAE (Validation):", lgb_rae_val)
print("LightGBM RRSE (Validation):", lgb_rrse_val)
print("LightGBM CC (Validation):", lgb_cc_val)

## 9. Gradient Boosting Regressor HPT

In [ ]:
# Define the search space for hyperparameters
param_dist = {
    'n_estimators': Integer(100, 300),
    'max_depth': Integer(3, 5),
    'learning_rate': Real(0.01, 0.2, prior='uniform')
}

# Initialize Bayesian Optimization for Gradient Boosting Regressor
bayes_search_gbr = BayesSearchCV(
    GradientBoostingRegressor(random_state=42),  
    param_dist,
    scoring='neg_mean_squared_error',
    cv=5,
    n_iter=10,  
    random_state=42,
    n_jobs=-1, 
    verbose=1, 
    n_points=5,  
    refit=True  
)

# Perform Bayesian Optimization
bayes_search_gbr.fit(X_train, y_train)

# Get the best hyperparameters and model for Gradient Boosting Regressor
best_gbr = bayes_search_gbr.best_estimator_
best_params_gbr = bayes_search_gbr.best_params_

# Predict on validation set using Gradient Boosting Regressor
gbr_pred_val = best_gbr.predict(X_val)

# Calculate metrics for Gradient Boosting Regressor on validation set
gbr_rmse_val = np.sqrt(mean_squared_error(y_val, gbr_pred_val))
gbr_mae_val = mean_absolute_error(y_val, gbr_pred_val)
gbr_r2_val = r2_score(y_val, gbr_pred_val)
gbr_rae_val = mean_absolute_percentage_error(y_val, gbr_pred_val)
gbr_rrse_val = np.sqrt(np.sum((y_val - gbr_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
gbr_cc_val = np.corrcoef(y_val, gbr_pred_val)[0, 1]

# Print the best hyperparameters and evaluation metrics for Gradient Boosting Regressor on validation set
print("Best Hyperparameters for Gradient Boosting Regressor:", best_params_gbr)
print("Gradient Boosting RMSE (Validation):", gbr_rmse_val)
print("Gradient Boosting MAE (Validation):", gbr_mae_val)
print("Gradient Boosting R-squared (Validation):", gbr_r2_val)
print("Gradient Boosting RAE (Validation):", gbr_rae_val)
print("Gradient Boosting RRSE (Validation):", gbr_rrse_val)
print("Gradient Boosting CC (Validation):", gbr_cc_val)

## 10.Ridge Regression HPT

In [ ]:
# Define the hyperparameters and their possible values for Ridge Regression
param_grid_ridge = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
                    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']} 

# Initialize Ridge model
ridge = Ridge(max_iter=10000)

# Initialize Grid Search
grid_search_ridge = GridSearchCV(ridge, param_grid_ridge, scoring='neg_mean_squared_error', cv=5)

# Train the Grid Search
grid_search_ridge.fit(X_train, y_train)

# Get the best parameters and model for Ridge Regression
best_ridge = grid_search_ridge.best_estimator_

# Predict on validation set using Ridge Regression
ridge_pred_val = best_ridge.predict(X_val)

# Calculate metrics for Ridge Regression
ridge_rmse_val = np.sqrt(mean_squared_error(y_val, ridge_pred_val))
ridge_mae_val = mean_absolute_error(y_val, ridge_pred_val)
ridge_r2_val = r2_score(y_val, ridge_pred_val)
ridge_rae_val = mean_absolute_percentage_error(y_val, ridge_pred_val)
ridge_rrse_val = np.sqrt(np.sum((y_val - ridge_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
ridge_cc_val = np.corrcoef(y_val, ridge_pred_val)[0, 1]

# Print the best hyperparameters and evaluation metrics for Ridge Regression
print("Best Hyperparameters for Ridge Regression:", grid_search_ridge.best_params_)
print("Ridge Regression RMSE (Validation):", ridge_rmse_val)
print("Ridge Regression MAE (Validation):", ridge_mae_val)
print("Ridge Regression R-squared (Validation):", ridge_r2_val)
print("Ridge Regression RAE (Validation):", ridge_rae_val)
print("Ridge Regression RRSE (Validation):", ridge_rrse_val)
print("Ridge Regression CC (Validation):", ridge_cc_val)

# Hybrid models

## Hybrid modeling

## Hybrid model 1: Training

In [ ]:
hybrid1_pred_val = (rf_pred_val + lwlr_pred_val) / 2

# Evaluate the performance of Hybrid Model 1
hybrid1_rmse_val = np.sqrt(mean_squared_error(y_val, hybrid1_pred_val))
hybrid1_mae_val = mean_absolute_error(y_val, hybrid1_pred_val)
hybrid1_r2_val = r2_score(y_val, hybrid1_pred_val)
hybrid1_rae_val = mean_absolute_percentage_error(y_val, hybrid1_pred_val)
hybrid1_rrse_val = np.sqrt(np.sum((y_val - hybrid1_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
hybrid1_cc_val = np.corrcoef(y_val, hybrid1_pred_val)[0, 1]

print("Hybrid Model 1 (Random Forest + LWLR) RMSE (Validation):", hybrid1_rmse_val)
print("Hybrid Model 1 (Random Forest + LWLR) MAE (Validation):", hybrid1_mae_val)
print("Hybrid Model 1 (Random Forest + LWLR) R-squared (Validation):", hybrid1_r2_val)
print("Hybrid Model 1 (Random Forest + LWLR) RAE (Validation):", hybrid1_rae_val)
print("Hybrid Model 1 (Random Forest + LWLR) RRSE (Validation):", hybrid1_rrse_val)
print("Hybrid Model 1 (Random Forest + LWLR) CC (Validation):", hybrid1_cc_val)

## Hybrid model 1: Test

In [ ]:
# Predict on the test set using Random Forest Regressor
rf_pred_test = best_rf.predict(X_test)

# Predict on the test set using Locally Weighted Linear Regression
lwlr_pred_test = best_lwlr.predict(X_test)


hybrid1_pred_test = (rf_pred_test + lwlr_pred_test) / 2

# Calculate metrics for the hybrid model on test set
hybrid1_rmse_test = np.sqrt(mean_squared_error(y_test, hybrid1_pred_test))
hybrid1_mae_test = mean_absolute_error(y_test, hybrid1_pred_test)
hybrid1_r2_test = r2_score(y_test, hybrid1_pred_test)
hybrid1_rae_test = mean_absolute_percentage_error(y_test, hybrid1_pred_test)
hybrid1_rrse_test = np.sqrt(np.sum((y_test - hybrid1_pred_test)**2) / np.sum((y_test - np.mean(y_test))**2))
hybrid1_cc_test = np.corrcoef(y_test, hybrid1_pred_test)[0, 1]

# Print evaluation metrics for Hybrid Model 1
print("Hybrid Model 1 (Random Forest + LWLR) RMSE (Test):", hybrid1_rmse_test)
print("Hybrid Model 1 (Random Forest + LWLR) MAE (Test):", hybrid1_mae_test)
print("Hybrid Model 1 (Random Forest + LWLR) R-squared (Test):", hybrid1_r2_test)
print("Hybrid Model 1 (Random Forest + LWLR) RAE (Test):", hybrid1_rae_test)
print("Hybrid Model 1 (Random Forest + LWLR) RRSE (Test):", hybrid1_rrse_test)
print("Hybrid Model 1 (Random Forest + LWLR) CC (Test):", hybrid1_cc_test)

## Hybrid model 2: Training

In [ ]:
# Calculate predictions for Hybrid Model 2 (GBR + CatBoost) on validation set
hybrid2_pred_val = (best_lgb.predict(X_val) + best_catboost.predict(X_val)) / 2

# Evaluate the performance of Hybrid Model 2
hybrid2_rmse_val = np.sqrt(mean_squared_error(y_val, hybrid2_pred_val))
hybrid2_mae_val = mean_absolute_error(y_val, hybrid2_pred_val)
hybrid2_r2_val = r2_score(y_val, hybrid2_pred_val)
hybrid2_rae_val = mean_absolute_percentage_error(y_val, hybrid2_pred_val)
hybrid2_rrse_val = np.sqrt(np.sum((y_val - np.mean(y_val))**2) / np.sum((y_val - hybrid2_pred_val)**2))
hybrid2_cc_val = np.corrcoef(y_val, hybrid2_pred_val)[0, 1]

# Print evaluation metrics for Hybrid Model 2
print("Hybrid Model 2 (GBR + CatBoost) RMSE (Validation):", hybrid2_rmse_val)
print("Hybrid Model 2 (GBR + CatBoost) MAE (Validation):", hybrid2_mae_val)
print("Hybrid Model 2 (GBR + CatBoost) R-squared (Validation):", hybrid2_r2_val)
print("Hybrid Model 2 (GBR + CatBoost) RAE (Validation):", hybrid2_rae_val)
print("Hybrid Model 2 (GBR + CatBoost) RRSE (Validation):", hybrid2_rrse_val)
print("Hybrid Model 2 (GBR + CatBoost) CC (Validation):", hybrid2_cc_val)


## Hybrid model 2: Test

In [ ]:
# Predict on the test set using Gradient Boosting Regressor
gbr_pred_test = best_lgb.predict(X_test)

# Predict on the test set using CatBoost Regressor
catboost_pred_test = best_catboost.predict(X_test)


hybrid2_pred_test = (gbr_pred_test + catboost_pred_test) / 2

# Evaluate the performance of Hybrid Model 2 on the test set
hybrid2_rmse_test = np.sqrt(mean_squared_error(y_test, hybrid2_pred_test))
hybrid2_mae_test = mean_absolute_error(y_test, hybrid2_pred_test)
hybrid2_r2_test = r2_score(y_test, hybrid2_pred_test)
hybrid2_rae_test = mean_absolute_percentage_error(y_test, hybrid2_pred_test)
hybrid2_rrse_test = np.sqrt(np.sum((y_test - hybrid2_pred_test)**2) / np.sum((y_test - np.mean(y_test))**2))
hybrid2_cc_test = np.corrcoef(y_test, hybrid2_pred_test)[0, 1]

print("Hybrid Model 2 (GBR + CatBoost) RMSE (Test):", hybrid2_rmse_test)
print("Hybrid Model 2 (GBR + CatBoost) MAE (Test):", hybrid2_mae_test)
print("Hybrid Model 2 (GBR + CatBoost) R-squared (Test):", hybrid2_r2_test)
print("Hybrid Model 2 (GBR + CatBoost) RAE (Test):", hybrid2_rae_test)
print("Hybrid Model 2 (GBR + CatBoost) RRSE (Test):", hybrid2_rrse_test)
print("Hybrid Model 2 (GBR + CatBoost) CC (Test):", hybrid2_cc_test)

## Hybrid model 3: Training

In [ ]:
hybrid3_pred_val = (ridge_pred_val + lwlr_pred_val) / 2

# Calculate metrics for the hybrid model on validation set
hybrid3_rmse_val = np.sqrt(mean_squared_error(y_val, hybrid3_pred_val))
hybrid3_mae_val = mean_absolute_error(y_val, hybrid3_pred_val)
hybrid3_r2_val = r2_score(y_val, hybrid3_pred_val)
hybrid3_rae_val = mean_absolute_percentage_error(y_val, hybrid3_pred_val)
hybrid3_rrse_val = np.sqrt(np.sum((y_val - hybrid3_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
hybrid3_cc_val = np.corrcoef(y_val, hybrid3_pred_val)[0, 1]

print("Hybrid Model 3 (Ridge + LWLR) RMSE (Validation):", hybrid3_rmse_val)
print("Hybrid Model 3 (Ridge + LWLR) MAE (Validation):", hybrid3_mae_val)
print("Hybrid Model 3 (Ridge + LWLR) R-squared (Validation):", hybrid3_r2_val)
print("Hybrid Model 3 (Ridge + LWLR) RAE (Validation):", hybrid3_rae_val)
print("Hybrid Model 3 (Ridge + LWLR) RRSE (Validation):", hybrid3_rrse_val)
print("Hybrid Model 3 (Ridge + LWLR) CC (Validation):", hybrid3_cc_val)

## Hybrid model 3: Test

In [ ]:
# Predict on the test set using Ridge Regression
ridge_pred_test = best_ridge.predict(X_test)

# Predict on the test set using Locally Weighted Linear Regression
lwlr_pred_test = best_lwlr.predict(X_test)

hybrid3_pred_test = (ridge_pred_test + lwlr_pred_test) / 2

# Calculate metrics for the hybrid model 3 on test set
hybrid3_rmse_test = np.sqrt(mean_squared_error(y_test, hybrid3_pred_test))
hybrid3_mae_test = mean_absolute_error(y_test, hybrid3_pred_test)
hybrid3_r2_test = r2_score(y_test, hybrid3_pred_test)
hybrid3_rae_test = mean_absolute_percentage_error(y_test, hybrid3_pred_test)
hybrid3_rrse_test = np.sqrt(np.sum((y_test - hybrid3_pred_test)**2) / np.sum((y_test - np.mean(y_test))**2))
hybrid3_cc_test = np.corrcoef(y_test, hybrid3_pred_test)[0, 1]

# Print evaluation metrics for Hybrid Model 3
print("Hybrid Model 3 (Ridge + LWLR) RMSE (Test):", hybrid3_rmse_test)
print("Hybrid Model 3 (Ridge + LWLR) MAE (Test):", hybrid3_mae_test)
print("Hybrid Model 3 (Ridge + LWLR) R-squared (Test):", hybrid3_r2_test)
print("Hybrid Model 3 (Ridge + LWLR) RAE (Test):", hybrid3_rae_test)
print("Hybrid Model 3 (Ridge + LWLR) RRSE (Test):", hybrid3_rrse_test)
print("Hybrid Model 3 (Ridge + LWLR) CC (Test):", hybrid3_cc_test)

## Hybrid model 4: Training

In [ ]:
# Define weights for models
weight_rf = 0.4
weight_lgb = 0.3
weight_catboost = 0.3

# Combine predictions
hybrid4_pred_val = (weight_rf * rf_pred_val + weight_lgb * lgb_pred_val + weight_catboost * catboost_pred_val)

# Evaluate the performance of Hybrid Model 4
hybrid4_rmse_val = np.sqrt(mean_squared_error(y_val, hybrid4_pred_val))
hybrid4_mae_val = mean_absolute_error(y_val, hybrid4_pred_val)
hybrid4_r2_val = r2_score(y_val, hybrid4_pred_val)
hybrid4_rae_val = mean_absolute_percentage_error(y_val, hybrid4_pred_val)
hybrid4_rrse_val = np.sqrt(np.sum((y_val - hybrid4_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
hybrid4_cc_val = np.corrcoef(y_val, hybrid4_pred_val)[0, 1]

print("Hybrid Model 4 (RF + LightGBM + CatBoost) RMSE (Validation):", hybrid4_rmse_val)
print("Hybrid Model 4 (RF + LightGBM + CatBoost) MAE (Validation):", hybrid4_mae_val)
print("Hybrid Model 4 (RF + LightGBM + CatBoost) R-squared (Validation):", hybrid4_r2_val)
print("Hybrid Model 4 (RF + LightGBM + CatBoost) RAE (Validation):", hybrid4_rae_val)
print("Hybrid Model 4 (RF + LightGBM + CatBoost) RRSE (Validation):", hybrid4_rrse_val)
print("Hybrid Model 4 (RF + LightGBM + CatBoost) CC (Validation):", hybrid4_cc_val)

## Hybrid model 4: Test

In [ ]:
# Define weights for models
weight_rf = 0.4
weight_lgb = 0.3
weight_catboost = 0.3


rf_pred_test = best_rf.predict(X_test)  
lgb_pred_test = best_lgb.predict(X_test) 
catboost_pred_test = best_catboost.predict(X_test)

# Combine predictions for the test set
hybrid4_pred_test = (weight_rf * rf_pred_test + weight_lgb * lgb_pred_test + weight_catboost * catboost_pred_test)

# Evaluate the performance of Hybrid Model 4 on the test set
hybrid4_rmse_test = np.sqrt(mean_squared_error(y_test, hybrid4_pred_test))
hybrid4_mae_test = mean_absolute_error(y_test, hybrid4_pred_test)
hybrid4_r2_test = r2_score(y_test, hybrid4_pred_test)
hybrid4_rae_test = mean_absolute_percentage_error(y_test, hybrid4_pred_test)
hybrid4_rrse_test = np.sqrt(np.sum((y_test - hybrid4_pred_test)**2) / np.sum((y_test - np.mean(y_test))**2))
hybrid4_cc_test = np.corrcoef(y_test, hybrid4_pred_test)[0, 1]

print("Hybrid Model 4 (RF + LightGBM + CatBoost) RMSE (Test):", hybrid4_rmse_test)
print("Hybrid Model 4 (RF + LightGBM + CatBoost) MAE (Test):", hybrid4_mae_test)
print("Hybrid Model 4 (RF + LightGBM + CatBoost) R-squared (Test):", hybrid4_r2_test)
print("Hybrid Model 4 (RF + LightGBM + CatBoost) RAE (Test):", hybrid4_rae_test)
print("Hybrid Model 4 (RF + LightGBM + CatBoost) RRSE (Test):", hybrid4_rrse_test)
print("Hybrid Model 4 (RF + LightGBM + CatBoost) CC (Test):", hybrid4_cc_test)

## Hybrid model 5: Training

In [ ]:
hybrid5_pred_val = (rf_pred_val + gpr_pred_val + lgb_pred_val) / 3

# Calculate metrics for the hybrid model 5 on validation set
hybrid5_rmse_val = np.sqrt(mean_squared_error(y_val, hybrid5_pred_val))
hybrid5_mae_val = mean_absolute_error(y_val, hybrid5_pred_val)
hybrid5_r2_val = r2_score(y_val, hybrid5_pred_val)
hybrid5_rae_val = mean_absolute_percentage_error(y_val, hybrid5_pred_val)
hybrid5_rrse_val = np.sqrt(np.sum((y_val - hybrid5_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
hybrid5_cc_val = np.corrcoef(y_val, hybrid5_pred_val)[0, 1]

print("Hybrid Model 5 (RF + GPR + LightGBM) RMSE:", hybrid5_rmse_val)
print("Hybrid Model 5 (RF + GPR + LightGBM) MAE:", hybrid5_mae_val)
print("Hybrid Model 5 (RF + GPR + LightGBM) R-squared:", hybrid5_r2_val)
print("Hybrid Model 5 (RF + GPR + LightGBM) RAE:", hybrid5_rae_val)
print("Hybrid Model 5 (RF + GPR + LightGBM) RRSE:", hybrid5_rrse_val)
print("Hybrid Model 5 (RF + GPR + LightGBM) CC:", hybrid5_cc_val)

## Hybrid model 5: Test

In [ ]:
gpr_pred_test = best_gpr.predict(X_test)

hybrid5_pred_test = (rf_pred_test + gpr_pred_test + lgb_pred_test) / 3

# Calculate metrics for the hybrid model 5 on the test set
hybrid5_rmse_test = np.sqrt(mean_squared_error(y_test, hybrid5_pred_test))
hybrid5_mae_test = mean_absolute_error(y_test, hybrid5_pred_test)
hybrid5_r2_test = r2_score(y_test, hybrid5_pred_test)
hybrid5_rae_test = mean_absolute_percentage_error(y_test, hybrid5_pred_test)
hybrid5_rrse_test = np.sqrt(np.sum((y_test - hybrid5_pred_test)**2) / np.sum((y_test - np.mean(y_test))**2))
hybrid5_cc_test = np.corrcoef(y_test, hybrid5_pred_test)[0, 1]

print("Hybrid Model 5 (RF + GPR + LightGBM) RMSE (Test):", hybrid5_rmse_test)
print("Hybrid Model 5 (RF + GPR + LightGBM) MAE (Test):", hybrid5_mae_test)
print("Hybrid Model 5 (RF + GPR + LightGBM) R-squared (Test):", hybrid5_r2_test)
print("Hybrid Model 5 (RF + GPR + LightGBM) RAE (Test):", hybrid5_rae_test)
print("Hybrid Model 5 (RF + GPR + LightGBM) RRSE (Test):", hybrid5_rrse_test)
print("Hybrid Model 5 (RF + GPR + LightGBM) CC (Test):", hybrid5_cc_test)

## Hybrid model 6: Training

In [ ]:
hybrid6_pred_val = (knn_pred + xgb_pred_val + catboost_pred_val) / 3

# Calculate metrics for the hybrid model 6 on validation set
hybrid6_rmse_val = np.sqrt(mean_squared_error(y_val, hybrid6_pred_val))
hybrid6_mae_val = mean_absolute_error(y_val, hybrid6_pred_val)
hybrid6_r2_val = r2_score(y_val, hybrid6_pred_val)
hybrid6_rae_val = mean_absolute_percentage_error(y_val, hybrid6_pred_val)
hybrid6_rrse_val = np.sqrt(np.sum((y_val - hybrid6_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
hybrid6_cc_val = np.corrcoef(y_val, hybrid6_pred_val)[0, 1]

print("Hybrid Model 6 (KNN + XGB + CatBoost) RMSE (Validation):", hybrid6_rmse_val)
print("Hybrid Model 6 (KNN + XGB + CatBoost) MAE (Validation):", hybrid6_mae_val)
print("Hybrid Model 6 (KNN + XGB + CatBoost) R-squared (Validation):", hybrid6_r2_val)
print("Hybrid Model 6 (KNN + XGB + CatBoost) RAE (Validation):", hybrid6_rae_val)
print("Hybrid Model 6 (KNN + XGB + CatBoost) RRSE (Validation):", hybrid6_rrse_val)
print("Hybrid Model 6 (KNN + XGB + CatBoost) CC (Validation):", hybrid6_cc_val)

## Hybrid model 6: Test

In [ ]:
best_knn_pred_test = best_knn.predict(X_test)
best_xgb_pred_test = best_xgb.predict(X_test)
best_catboost_pred_test = best_catboost.predict(X_test)

hybrid6_pred_test = (best_knn_pred_test + best_xgb_pred_test + best_catboost_pred_test) / 3

# Calculate metrics for the hybrid model 6 on test set
hybrid6_rmse_test = np.sqrt(mean_squared_error(y_test, hybrid6_pred_test))
hybrid6_mae_test = mean_absolute_error(y_test, hybrid6_pred_test)
hybrid6_r2_test = r2_score(y_test, hybrid6_pred_test)
hybrid6_rae_test = mean_absolute_percentage_error(y_test, hybrid6_pred_test)
hybrid6_rrse_test = np.sqrt(np.sum((y_test - hybrid6_pred_test)**2) / np.sum((y_test - np.mean(y_test))**2))
hybrid6_cc_test = np.corrcoef(y_test, hybrid6_pred_test)[0, 1]

print("Hybrid Model 6 (KNN + XGB + CatBoost) RMSE (Test):", hybrid6_rmse_test)
print("Hybrid Model 6 (KNN + XGB + CatBoost) MAE (Test):", hybrid6_mae_test)
print("Hybrid Model 6 (KNN + XGB + CatBoost) R-squared (Test):", hybrid6_r2_test)
print("Hybrid Model 6 (KNN + XGB + CatBoost) RAE (Test):", hybrid6_rae_test)
print("Hybrid Model 6 (KNN + XGB + CatBoost) RRSE (Test):", hybrid6_rrse_test)
print("Hybrid Model 6 (KNN + XGB + CatBoost) CC (Test):", hybrid6_cc_test)

## Hybrid model 7: Training

In [ ]:
hybrid7_pred_val = (rf_pred_val + lwlr_pred_val + gpr_pred_val + wknn_pred_val) / 4

# Calculate metrics for the hybrid model 7 on validation set
hybrid7_rmse_val = np.sqrt(mean_squared_error(y_val, hybrid7_pred_val))
hybrid7_mae_val = mean_absolute_error(y_val, hybrid7_pred_val)
hybrid7_r2_val = r2_score(y_val, hybrid7_pred_val)
hybrid7_rae_val = mean_absolute_percentage_error(y_val, hybrid7_pred_val)
hybrid7_rrse_val = np.sqrt(np.sum((y_val - hybrid7_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
hybrid7_cc_val = np.corrcoef(y_val, hybrid7_pred_val)[0, 1]

print("Hybrid Model 7 (RF + LWLR + GPR + WKNN) RMSE (Validation):", hybrid7_rmse_val)
print("Hybrid Model 7 (RF + LWLR + GPR + WKNN) MAE (Validation):", hybrid7_mae_val)
print("Hybrid Model 7 (RF + LWLR + GPR + WKNN) R-squared (Validation):", hybrid7_r2_val)
print("Hybrid Model 7 (RF + LWLR + GPR + WKNN) RAE (Validation):", hybrid7_rae_val)
print("Hybrid Model 7 (RF + LWLR + GPR + WKNN) RRSE (Validation):", hybrid7_rrse_val)
print("Hybrid Model 7 (RF + LWLR + GPR + WKNN) CC (Validation):", hybrid7_cc_val)

## Hybrid model 7: Test

In [ ]:
# Predict on the test set using Random Forest Regressor
rf_pred_test = best_rf.predict(X_test)

# Predict on the test set using Locally Weighted Linear Regression
lwlr_pred_test = best_lwlr.predict(X_test)

# Predict on the test set using Gaussian Process Regressor
gpr_pred_test = best_gpr.predict(X_test)

# Predict on the test set using Weighted k-Nearest Neighbors Regressor
wknn_pred_test = best_wknn.predict(X_test)

hybrid7_pred_test = (rf_pred_test + lwlr_pred_test + gpr_pred_test + wknn_pred_test) / 4

# Calculate metrics for the hybrid model 7 on test set
hybrid7_rmse_test = np.sqrt(mean_squared_error(y_test, hybrid7_pred_test))
hybrid7_mae_test = mean_absolute_error(y_test, hybrid7_pred_test)
hybrid7_r2_test = r2_score(y_test, hybrid7_pred_test)
hybrid7_rae_test = mean_absolute_percentage_error(y_test, hybrid7_pred_test)
hybrid7_rrse_test = np.sqrt(np.sum((y_test - hybrid7_pred_test)**2) / np.sum((y_test - np.mean(y_test))**2))
hybrid7_cc_test = np.corrcoef(y_test, hybrid7_pred_test)[0, 1]

# Print evaluation metrics for Hybrid Model 7
print("Hybrid Model 7 (RF + LWLR + GPR + WKNN) RMSE (Test):", hybrid7_rmse_test)
print("Hybrid Model 7 (RF + LWLR + GPR + WKNN) MAE (Test):", hybrid7_mae_test)
print("Hybrid Model 7 (RF + LWLR + GPR + WKNN) R-squared (Test):", hybrid7_r2_test)
print("Hybrid Model 7 (RF + LWLR + GPR + WKNN) RAE (Test):", hybrid7_rae_test)
print("Hybrid Model 7 (RF + LWLR + GPR + WKNN) RRSE (Test):", hybrid7_rrse_test)
print("Hybrid Model 7 (RF + LWLR + GPR + WKNN) CC (Test):", hybrid7_cc_test)

## Hybrid model 8: Training

In [ ]:
hybrid8_pred_val = (xgb_pred_val + catboost_pred_val + lgb_pred_val + ridge_pred_val) / 4

# Calculate metrics for the hybrid model on validation set
hybrid8_rmse_val = np.sqrt(mean_squared_error(y_val, hybrid8_pred_val))
hybrid8_mae_val = mean_absolute_error(y_val, hybrid8_pred_val)
hybrid8_r2_val = r2_score(y_val, hybrid8_pred_val)
hybrid8_rae_val = mean_absolute_percentage_error(y_val, hybrid8_pred_val)
hybrid8_rrse_val = np.sqrt(np.sum((y_val - hybrid8_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
hybrid8_cc_val = np.corrcoef(y_val, hybrid8_pred_val)[0, 1]

# Print evaluation metrics for Hybrid Model 8
print("Hybrid Model 8 (XGB + CatBoost + GBR + Ridge) RMSE (Validation):", hybrid8_rmse_val)
print("Hybrid Model 8 (XGB + CatBoost + GBR + Ridge) MAE (Validation):", hybrid8_mae_val)
print("Hybrid Model 8 (XGB + CatBoost + GBR + Ridge) R-squared (Validation):", hybrid8_r2_val)
print("Hybrid Model 8 (XGB + CatBoost + GBR + Ridge) RAE (Validation):", hybrid8_rae_val)
print("Hybrid Model 8 (XGB + CatBoost + GBR + Ridge) RRSE (Validation):", hybrid8_rrse_val)
print("Hybrid Model 8 (XGB + CatBoost + GBR + Ridge) CC (Validation):", hybrid8_cc_val)

## Hybrid model 8: Test

In [ ]:
# Predict on the test set using the hybrid model
hybrid8_pred_test = (best_xgb.predict(X_test) + best_catboost.predict(X_test) + best_lgb.predict(X_test) + best_ridge.predict(X_test)) / 4

# Calculate metrics for the hybrid model 8 on the test set
hybrid8_rmse_test = np.sqrt(mean_squared_error(y_test, hybrid8_pred_test))
hybrid8_mae_test = mean_absolute_error(y_test, hybrid8_pred_test)
hybrid8_r2_test = r2_score(y_test, hybrid8_pred_test)
hybrid8_rae_test = mean_absolute_percentage_error(y_test, hybrid8_pred_test)
hybrid8_rrse_test = np.sqrt(np.sum((y_test - hybrid8_pred_test)**2) / np.sum((y_test - np.mean(y_test))**2))
hybrid8_cc_test = np.corrcoef(y_test, hybrid8_pred_test)[0, 1]

# Print evaluation metrics for the hybrid model 8 on the test set
print("Hybrid Model 5 (XGB + CatBoost + GBR + Ridge) RMSE (Test):", hybrid8_rmse_test)
print("Hybrid Model 5 (XGB + CatBoost + GBR + Ridge) MAE (Test):", hybrid8_mae_test)
print("Hybrid Model 5 (XGB + CatBoost + GBR + Ridge) R-squared (Test):", hybrid8_r2_test)
print("Hybrid Model 5 (XGB + CatBoost + GBR + Ridge) RAE (Test):", hybrid8_rae_test)
print("Hybrid Model 5 (XGB + CatBoost + GBR + Ridge) RRSE (Test):", hybrid8_rrse_test)
print("Hybrid Model 5 (XGB + CatBoost + GBR + Ridge) CC (Test):", hybrid8_cc_test)

## Hybrid model 9: Training

In [ ]:
estimators = [
    ('rf', best_rf),
    ('gpr', best_gpr),
    ('lgb', best_lgb),
    ('xgb', best_xgb),
    ('catboost', best_catboost)
]

# Define the stacking model with a meta regressor 
stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=best_lgb,  
    cv=5  
)

# Fit the stacking model on the training data
stacking_model.fit(X_train, y_train)

# Predict on the validation set using the stacking model
hybrid9_pred_val = stacking_model.predict(X_val)

# Calculate metrics for the hybrid model 9 on the validation set
hybrid9_rmse_val = np.sqrt(mean_squared_error(y_val, hybrid9_pred_val))
hybrid9_mae_val = mean_absolute_error(y_val, hybrid9_pred_val)
hybrid9_r2_val = r2_score(y_val, hybrid9_pred_val)
hybrid9_rae_val = mean_absolute_percentage_error(y_val, hybrid9_pred_val)
hybrid9_rrse_val = np.sqrt(np.sum((y_val - hybrid9_pred_val)**2) / np.sum((y_val - np.mean(y_val))**2))
hybrid9_cc_val = np.corrcoef(y_val, hybrid9_pred_val)[0, 1]

print("Hybrid Model 9 RMSE (Validation):", hybrid9_rmse_val)
print("Hybrid Model 9 MAE (Validation):", hybrid9_mae_val)
print("Hybrid Model 9 R-squared (Validation):", hybrid9_r2_val)
print("Hybrid Model 9 RAE (Validation):", hybrid9_rae_val)
print("Hybrid Model 9 RRSE (Validation):", hybrid9_rrse_val)
print("Hybrid Model 9 CC (Validation):", hybrid9_cc_val)

## Hybrid model 9: Test

In [ ]:
# Predict on the test set using the stacking model
hybrid9_pred_test = stacking_model.predict(X_test)

# Calculate metrics for the hybrid model 9 on test set
hybrid9_rmse_test = np.sqrt(mean_squared_error(y_test, hybrid9_pred_test))
hybrid9_mae_test = mean_absolute_error(y_test, hybrid9_pred_test)
hybrid9_r2_test = r2_score(y_test, hybrid9_pred_test)
hybrid9_rae_test = mean_absolute_percentage_error(y_test, hybrid9_pred_test)
hybrid9_rrse_test = np.sqrt(np.sum((y_test - hybrid9_pred_test)**2) / np.sum((y_test - np.mean(y_test))**2))
hybrid9_cc_test = np.corrcoef(y_test, hybrid9_pred_test)[0, 1]

print("Hybrid Model 9 RMSE (Test):", hybrid9_rmse_test)
print("Hybrid Model 9 MAE (Test):", hybrid9_mae_test)
print("Hybrid Model 9 R-squared (Test):", hybrid9_r2_test)
print("Hybrid Model 9 RAE (Test):", hybrid9_rae_test)
print("Hybrid Model 9 RRSE (Test):", hybrid9_rrse_test)
print("Hybrid Model 9 CC (Test):", hybrid9_cc_test)

## Hybrid model 10: Training

In [ ]:
# Define the weights for each model
weight_rf = 0.2
weight_lwlr = 0.2
weight_gpr = 0.1
weight_wknn = 0.1
weight_knn = 0.1
weight_xgb = 0.05
weight_catboost = 0.05
weight_lgb = 0.05
weight_gbr = 0.05
weight_ridge = 0.05

# Create hybrid predictions
hybrid10_pred_val = (
    weight_rf * rf_pred_val +
    weight_lwlr * lwlr_pred_val +
    weight_gpr * gpr_pred_val +
    weight_wknn * wknn_pred_val +
    weight_knn * knn_pred + 
    weight_xgb * xgb_pred_val +
    weight_catboost * catboost_pred_val +
    weight_gbr * gbr_pred_val +
    weight_lgb * lgb_pred_val +
    weight_ridge * ridge_pred_val  
)

# Calculate metrics for the hybrid model 10 on the validation set
hybrid10_rmse_val = mean_squared_error(y_val, hybrid10_pred_val, squared=False)
hybrid10_mae_val = mean_absolute_error(y_val, hybrid10_pred_val)
hybrid10_r2_val = r2_score(y_val, hybrid10_pred_val)
hybrid10_rae_val = hybrid10_mae_val / np.mean(np.abs(y_val - np.mean(y_val)))
hybrid10_rrse_val = hybrid10_rmse_val / np.sqrt(np.mean((y_val - np.mean(y_val))**2))
hybrid10_cc_val = np.corrcoef(y_val, hybrid10_pred_val)[0, 1]

# Print evaluation metrics for the hybrid model 10 on the validation set
print("Hybrid Model 10 (RF + LWLR + GPR + WKNN + KNN + XGB + CatBoost + LGB + GBR + Ridge) RMSE (Validation):", hybrid10_rmse_val)
print("Hybrid Model 10 (RF + LWLR + GPR + WKNN + KNN + XGB + CatBoost + LGB + GBR + Ridge) MAE (Validation):", hybrid10_mae_val)
print("Hybrid Model 10 (RF + LWLR + GPR + WKNN + KNN + XGB + CatBoost + LGB + GBR + Ridge) R-squared (Validation):", hybrid10_r2_val)
print("Hybrid Model 10 (RF + LWLR + GPR + WKNN + KNN + XGB + CatBoost + LGB + GBR + Ridge) RAE (Validation):", hybrid10_rae_val)
print("Hybrid Model 10 (RF + LWLR + GPR + WKNN + KNN + XGB + CatBoost + LGB + GBR + Ridge) RRSE (Validation):", hybrid10_rrse_val)
print("Hybrid Model 10 (RF + LWLR + GPR + WKNN + KNN + XGB + CatBoost + LGB + GBR + Ridge) CC (Validation):", hybrid10_cc_val)

## Hybrid model 10: Test

In [ ]:
# Define the weights for each model
weight_rf = 0.2
weight_lwlr = 0.2
weight_gpr = 0.1
weight_wknn = 0.1
weight_knn = 0.1
weight_xgb = 0.05
weight_catboost = 0.05
weight_lgb = 0.05
weight_gbr = 0.05
weight_ridge = 0.05

# Create hybrid predictions
hybrid10_pred_test = (
    weight_rf * rf_pred_test +
    weight_lwlr * lwlr_pred_test +
    weight_gpr * gpr_pred_test +
    weight_wknn * wknn_pred_test +
    weight_knn * knn_pred + 
    weight_xgb * xgb_pred_test +
    weight_catboost * catboost_pred_test +
    weight_gbr * gbr_pred_test +
    weight_lgb * lgb_pred_test +
    weight_ridge * ridge_pred_test  
)


rf_pred_test = best_rf.predict(X_test)
lwlr_pred_test = best_lwlr.predict(X_test)
gpr_pred_test = best_gpr.predict(X_test)
wknn_pred_test=best_wknn.predict(X_test)
knn_pred_test=best_knn.predict(X_test)
xgb_pred_test = best_xgb.predict(X_test)
catboost_pred_test = best_catboost.predict(X_test)
gbr_pred_test = best_gbr.predict(X_test)
lgb_pred_test = best_lgb.predict(X_test)
ridge_pred_test = best_ridge.predict(X_test)

# Calculate metrics for the hybrid model 10 on the testidation set
hybrid10_rmse_test = mean_squared_error(y_test, hybrid10_pred_test, squared=False)
hybrid10_mae_test = mean_absolute_error(y_test, hybrid10_pred_test)
hybrid10_r2_test = r2_score(y_test, hybrid10_pred_test)
hybrid10_rae_test = hybrid10_mae_test / np.mean(np.abs(y_test - np.mean(y_test)))
hybrid10_rrse_test = hybrid10_rmse_test / np.sqrt(np.mean((y_test - np.mean(y_test))**2))
hybrid10_cc_test = np.corrcoef(y_test, hybrid10_pred_test)[0, 1]

# Print etestuation metrics for the hybrid model 10 on the testidation set
print("Hybrid Model 10 (RF + LWLR + GPR + WKNN + KNN + XGB + CatBoost + LGB + GBR + Ridge) RMSE (testidation):", hybrid10_rmse_test)
print("Hybrid Model 10 (RF + LWLR + GPR + WKNN + KNN + XGB + CatBoost + LGB + GBR + Ridge) MAE (testidation):", hybrid10_mae_test)
print("Hybrid Model 10 (RF + LWLR + GPR + WKNN + KNN + XGB + CatBoost + LGB + GBR + Ridge) R-squared (testidation):", hybrid10_r2_test)
print("Hybrid Model 10 (RF + LWLR + GPR + WKNN + KNN + XGB + CatBoost + LGB + GBR + Ridge) RAE (testidation):", hybrid10_rae_test)
print("Hybrid Model 10 (RF + LWLR + GPR + WKNN + KNN + XGB + CatBoost + LGB + GBR + Ridge) RRSE (testidation):", hybrid10_rrse_test)
print("Hybrid Model 10 (RF + LWLR + GPR + WKNN + KNN + XGB + CatBoost + LGB + GBR + Ridge) CC (testidation):", hybrid10_cc_test)

# Visualization

## Scatter plot: Training: Actual vs. Predicted

In [ ]:
# Create a scatter plot
plt.figure(figsize=(10, 8))

plt.scatter(y_val, hybrid1_pred_val, color='red', edgecolors='black', s=40, linewidths=1.5, label='Hybrid Model 1')
plt.scatter(y_val, hybrid2_pred_val, color='green', edgecolors='red', s=70, linewidths=1.5, alpha=0.2, label='Hybrid Model 2')
plt.scatter(y_val, hybrid3_pred_val, color='blue', edgecolors='black', s=50, linewidths=1.5,alpha=0.1, label='Hybrid Model 3')
plt.scatter(y_val, hybrid4_pred_val, color='orange', edgecolors='black', s=70, linewidths=1.5, label='Hybrid Model 4')
plt.scatter(y_val,  hybrid5_pred_val, color='purple', edgecolors='blue', s=50, linewidths=1.5, label='Hybrid Model 5')
plt.scatter(y_val, hybrid6_pred_val, color='pink', edgecolors='black', s=70, linewidths=1.5,alpha=0.3, label='Hybrid Model 6')
plt.scatter(y_val,  hybrid7_pred_val, color='purple', edgecolors='black', s=50, linewidths=1.5, label='Hybrid Model 7')
plt.scatter(y_val, hybrid8_pred_val, color='cyan', edgecolors='black', s=40, linewidths=1.5,alpha=0.2, label='Hybrid Model 8')
plt.scatter(y_val, hybrid9_pred_val, color='#FF00FF', edgecolors='black', s=50, linewidths=1.5, label='Hybrid Model 9')
plt.scatter(y_val, hybrid10_pred_val, color='#00FFFF', edgecolors='black', s=50, linewidths=2,alpha=0.1 ,label='Hybrid Model 10')

plt.plot([2, 6], [2, 6], color='black')
plt.xlabel('Actual Values (val)')
plt.ylabel('Predicted Values (val)')
plt.title('10 Hybrid Models: Actual vs. Predicted(val)')
plt.grid(False)
plt.legend()
plt.show()

## Scatter plot: Test: Actual vs. Predicted

In [ ]:
# Create a scatter plot
plt.figure(figsize=(10, 8))

plt.scatter(y_test, hybrid1_pred_test, color='red', edgecolors='black', s=40, linewidths=1.5, label='Hybrid Model 1')
plt.scatter(y_test, hybrid2_pred_test, color='green', edgecolors='red', s=70, linewidths=1.5, alpha=0.2, label='Hybrid Model 2')
plt.scatter(y_test, hybrid3_pred_test, color='blue', edgecolors='black', s=50, linewidths=1.5,alpha=0.1, label='Hybrid Model 3')
plt.scatter(y_test, hybrid4_pred_test, color='orange', edgecolors='black', s=70, linewidths=1.5, label='Hybrid Model 4')
plt.scatter(y_test,  hybrid5_pred_test, color='purple', edgecolors='blue', s=50, linewidths=1.5, label='Hybrid Model 5')
plt.scatter(y_test, hybrid6_pred_test, color='pink', edgecolors='black', s=70, linewidths=1.5,alpha=0.3, label='Hybrid Model 6')
plt.scatter(y_test,  hybrid7_pred_test, color='purple', edgecolors='black', s=50, linewidths=1.5, label='Hybrid Model 7')
plt.scatter(y_test, hybrid8_pred_test, color='cyan', edgecolors='black', s=40, linewidths=1.5,alpha=0.2, label='Hybrid Model 8')
plt.scatter(y_test, hybrid9_pred_test, color='#FF00FF', edgecolors='black', s=50, linewidths=1.5, label='Hybrid Model 9')
plt.scatter(y_test, hybrid10_pred_test, color='#00FFFF', edgecolors='black', s=50, linewidths=2,alpha=0.1 ,label='Hybrid Model 10')

plt.plot([2, 6], [2, 6], color='black')
plt.xlabel('Actual Values (Test)')
plt.ylabel('Predicted Values (Test)')
plt.title('10 Hybrid Models: Actual vs. Predicted(Test)')
plt.grid(False)
plt.legend()
plt.show()

In [ ]:
# actual and predicted values for test set

hybrid1_test = pd.DataFrame({'Actual Values': y_test.values, 'Predicted Values': hybrid1_pred_test})
print("\nTest Set:")
hybrid1_test

In [ ]:
# Define the models and their corresponding predictions
models_predictions = {
    'Hybrid Model 1': hybrid1_pred_test,
    'Hybrid Model 2': hybrid2_pred_test,
    'Hybrid Model 3': hybrid3_pred_test,
    'Hybrid Model 4': hybrid4_pred_test,
    'Hybrid Model 5': hybrid5_pred_test,
    'Hybrid Model 6': hybrid6_pred_test,
    'Hybrid Model 7': hybrid7_pred_test,
    'Hybrid Model 8': hybrid8_pred_test,
    'Hybrid Model 9': hybrid9_pred_test,
    'Hybrid Model 10': hybrid10_pred_test
}

# Create a DataFrame for the actual values (y_test)
actual_values = pd.DataFrame({'Actual Values': y_test.values})

# Create a plot
plt.figure(figsize=(12, 6))

# Plot the actual values
plt.plot(actual_values.index, actual_values['Actual Values'], label='Actual Values', linestyle='-', marker='o', markersize=3)

# Plot the predicted values for each model
for model_name, predictions in models_predictions.items():
    plt.plot(actual_values.index, predictions, label=model_name, linestyle='-', marker='o', markersize=3, alpha=0.7)

plt.xlabel('Date')
plt.ylabel('Dinajpur-34 (Dinajpur Sadar)')
plt.title("Actual vs. Predicted Values Over Index")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# Time series plot

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format="%d.%m.%Y")

# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

# Create a time series plot
plt.figure(figsize=(10, 8))

# Plotting the time series data for each hybrid model

plt.plot(df.index, hybrid1_pred_val, color='red', linewidth=1.5, label='Hybrid Model 1')
plt.plot(df.index, hybrid2_pred_val, color='green', linewidth=1.5, alpha=0.2, label='Hybrid Model 2')
plt.plot(df.index, hybrid3_pred_val, color='blue', linewidth=1.5, alpha=0.3, label='Hybrid Model 3')
plt.plot(df.index, hybrid4_pred_val, color='orange', linewidth=1.5, label='Hybrid Model 4')
plt.plot(df.index, hybrid5_pred_val, color='purple', linewidth=1.5, label='Hybrid Model 5')
plt.plot(df.index, hybrid6_pred_val, color='pink', linewidth=1.5, alpha=0.3, label='Hybrid Model 6')
plt.plot(df.index, hybrid7_pred_val, color='purple', linewidth=1.5, label='Hybrid Model 7')
plt.plot(df.index, hybrid8_pred_val, color='cyan', linewidth=1.5, alpha=0.2, label='Hybrid Model 8')
plt.plot(df.index, hybrid9_pred_val, color='#00FFFF', linewidth=2, alpha=0.1, label='Hybrid Model 9')
plt.plot(df.index, hybrid10_pred_val, color='#00FFFF', linewidth=2, alpha=0.1, label='Hybrid Model 10')

# Customize the plot
plt.xlabel('Date')
plt.ylabel('Predicted Values (Training)')
plt.title('10 Hybrid Models: Actual vs. Predicted (Training)')
plt.grid(False)
plt.legend()
plt.show()

In [ ]:
# Assuming you have the following variables:
actual_values_train = df_train['Actual Values']  # Replace df_train with your actual DataFrame
predicted_values_train = df_train['Predicted Values']  # Replace df_train with your actual DataFrame

# Create a time axis (x-axis) for training set
time_points_train = range(len(actual_values_train))

# Plot the actual values during training
plt.figure(figsize=(12, 6))
plt.plot(time_points_train, actual_values_train, label='Actual (Training)', color='blue')
plt.plot(time_points_train, predicted_values_train, label='Predicted (Training)', color='red')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Actual vs. Predicted Values (Training Stage)')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming df is your DataFrame with the time series data
# Convert the 'Date' column to a datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

# Create a time series plot
plt.figure(figsize=(10, 8))

# Plotting the time series data for each hybrid model
plt.plot(df.index, y_test, color='red', linewidth=1.5, label='Hybrid Model 1')
plt.plot(df.index, hybrid1_pred_test, color='red', linewidth=1.5, label='Hybrid Model 1')
plt.plot(df.index, hybrid2_pred_test, color='green', linewidth=1.5, alpha=0.2, label='Hybrid Model 2')
plt.plot(df.index, hybrid3_pred_test, color='blue', linewidth=1.5, alpha=0.3, label='Hybrid Model 3')
plt.plot(df.index, hybrid4_pred_test, color='orange', linewidth=1.5, label='Hybrid Model 4')
plt.plot(df.index, hybrid5_pred_test, color='purple', linewidth=1.5, label='Hybrid Model 5')
plt.plot(df.index, hybrid6_pred_test, color='pink', linewidth=1.5, alpha=0.3, label='Hybrid Model 6')
plt.plot(df.index, hybrid7_pred_test, color='purple', linewidth=1.5, label='Hybrid Model 7')
plt.plot(df.index, hybrid8_pred_test, color='cyan', linewidth=1.5, alpha=0.2, label='Hybrid Model 8')
plt.plot(df.index, hybrid9_pred_test, color='#00FFFF', linewidth=2, alpha=0.1, label='Hybrid Model 9')
plt.plot(df.index, hybrid10_pred_test, color='#00FFFF', linewidth=2, alpha=0.1, label='Hybrid Model 10')

# Customize the plot
plt.xlabel('Date')
plt.ylabel('Predicted testues (Training)')
plt.title('10 Hybrid Models: Actual vs. Predicted (Training)')
plt.grid(False)
plt.legend()
plt.show()

# Density Plot

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Assuming df is your DataFrame with the time series data
# Convert the 'Date' column to a datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

# Create a figure
plt.figure(figsize=(10, 8))

# Plotting density plots for each hybrid model
sns.kdeplot(y_val, color='red', label='Hybrid Model 1')
sns.kdeplot(hybrid1_pred_val, color='red', label='Hybrid Model 1')
sns.kdeplot(hybrid2_pred_val, color='green', label='Hybrid Model 2')
sns.kdeplot(hybrid3_pred_val, color='blue', label='Hybrid Model 3')
sns.kdeplot(hybrid4_pred_val, color='orange', label='Hybrid Model 4')
sns.kdeplot(hybrid5_pred_val, color='purple', label='Hybrid Model 5')
sns.kdeplot(hybrid6_pred_val, color='pink', label='Hybrid Model 6')
sns.kdeplot(hybrid7_pred_val, color='purple', label='Hybrid Model 7')
sns.kdeplot(hybrid8_pred_val, color='cyan', label='Hybrid Model 8')
sns.kdeplot(hybrid9_pred_val, color='#00FFFF', label='Hybrid Model 9')
sns.kdeplot(hybrid10_pred_val, color='#00FFFF', label='Hybrid Model 10')

# Customize the plot
plt.xlabel('Values')
plt.ylabel('Density')
plt.title('Density Plot of Hybrid Models')
plt.grid(False)
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Assuming df is your DataFrame with the time series data
# Convert the 'Date' column to a datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

# Create a figure
plt.figure(figsize=(10, 8))

# Plotting density plots for each hybrid model
sns.kdeplot(y_test, color='red', label='Hybrid Model 1')
sns.kdeplot(hybrid1_pred_test, color='red', label='Hybrid Model 1')
sns.kdeplot(hybrid2_pred_test, color='green', label='Hybrid Model 2')
sns.kdeplot(hybrid3_pred_test, color='blue', label='Hybrid Model 3')
sns.kdeplot(hybrid4_pred_test, color='orange', label='Hybrid Model 4')
sns.kdeplot(hybrid5_pred_test, color='purple', label='Hybrid Model 5')
sns.kdeplot(hybrid6_pred_test, color='pink', label='Hybrid Model 6')
sns.kdeplot(hybrid7_pred_test, color='purple', label='Hybrid Model 7')
sns.kdeplot(hybrid8_pred_test, color='cyan', label='Hybrid Model 8')
sns.kdeplot(hybrid9_pred_test, color='#00FFFF', label='Hybrid Model 9')
sns.kdeplot(hybrid10_pred_test, color='#00FFFF', label='Hybrid Model 10')

# Customize the plot
plt.xlabel('testues')
plt.ylabel('Density')
plt.title('Density Plot of Hybrid Models')
plt.grid(False)
plt.legend()
plt.show()

## Residual Plot

In [ ]:
import matplotlib.pyplot as plt

# Calculate residuals for Hybrid Model 1
residuals_hybrid1 = y_val - hybrid1_pred_val

# Create Residual Plot
plt.figure(figsize=(8, 6))
plt.scatter(hybrid1_pred_val, residuals_hybrid1, color='blue')
plt.title('Hybrid Model 1 Residual Plot')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='r', linestyle='-')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Calculate residuals for Hybrid Model 1
residuals_hybrid1 = y_test - hybrid1_pred_test

# Create Residual Plot
plt.figure(figsize=(8, 6))
plt.scatter(hybrid1_pred_test, residuals_hybrid1, color='blue')
plt.title('Hybrid Model 1 Residual Plot')
plt.xlabel('Predicted testues')
plt.ylabel('Residuals')
plt.axhline(y=0, color='r', linestyle='-')
plt.show()

# Densty plot /error distribution:

In [ ]:
import seaborn as sns

# Create Density Plot of Residuals for Hybrid Model 1
plt.figure(figsize=(8, 6))
sns.histplot(residuals_hybrid1, kde=True, color='blue')
plt.title('Density Plot of Residuals (Hybrid Model 1)')
plt.xlabel('Residuals')
plt.ylabel('Density')
plt.show()


# QQ Plot:

In [ ]:
import statsmodels.api as sm
import scipy.stats as stats

# Create QQ Plot for Hybrid Model 1
plt.figure(figsize=(8, 6))
sm.qqplot(residuals_hybrid1, line='45', fit=True, color='blue')
plt.title('QQ Plot (Hybrid Model 1)')
plt.show()